# Get the CSV

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv')
train_df.head()

In [ ]:
print(f"Total no of records : {len(train_df)}\nTotal : {len(train_df)//3}") #each record have three classes

In [ ]:
print(f"No of classes: {train_df['class'].nunique()},\nClasses: {train_df['class'].unique()}")

In [ ]:
train_dir = '../input/uw-madison-gi-tract-image-segmentation/train/'
test_dir = '../input/uw-madison-gi-tract-image-segmentation/test/'

In [ ]:
#check number of images
import glob, os
train_images = glob.glob(os.path.join(train_dir, "**", '*.png'), recursive=True)
print("Total No of Training Images:    ",len(train_images))
test_images = glob.glob(os.path.join(test_dir,"**", "*.png"), recursive = True)
print("Total No of Test Images:    ", len(test_images))

In [ ]:
def create_train_df(images_paths):
    df1 = pd.DataFrame()
    df = {}
    for each in images_paths:
        #get case information
        df["case_str"]=[each.split('_')[0].split('/')[-1]]
        df["case"]=[int(each.split('_')[0].split('/')[-1].replace("case",""))]
        #get day information
        df["day_str"]=[each.split('_')[1].split('/')[0]]
        df["day"]=[int(each.split('_')[1].split('/')[0].replace("day",""))]
        #get slice information
        df["slice"] = [each.split('_')[2]]
        #get image width and height
        df["height"] =[ int(each.split('_')[3])]
        df["width"] = [int(each.split('_')[4])]
        #path of image
        df["path"] =[ each]
        #id 
        df["id"] = [each.split('_')[0].split('/')[-1]+'_'+each.split('_')[1].split('/')[0]+'_slice_'+each.split('_')[2]]
        
        #append df1
        
        temp_df = pd.DataFrame.from_dict(df)
        
        df1=df1.append(temp_df, ignore_index=True)
       
        
    return df1
temp_df = create_train_df(train_images) #not required to perform already saved the preprocessed dataframe csv, going to use that one.....

In [ ]:
temp_df2 = train_df.merge(temp_df, on='id')

In [ ]:
temp_df2.head()

In [ ]:
def restructure(df):
    classes=df['class'].unique()
    classes_df = pd.DataFrame()
    for i in range(0, len(df), 3):
        print(i)
        dat_dict={}
        id_ = df.iloc[i]['id']
        s_cl_0 = list(df[(df['id']==id_) & (df['class']==classes[0])]['segmentation'])
        s_cl_1 = list(df[(df['id']==id_) & (df['class']==classes[1])]['segmentation'])
        s_cl_2 = list(df[(df['id']==id_) & (df['class']==classes[2])]['segmentation'])
        dat_dict['id']=id_        
        dat_dict[classes[0]]=s_cl_0
        dat_dict[classes[1]]=s_cl_1
        dat_dict[classes[2]]=s_cl_2
        dfa = pd.DataFrame.from_dict(dat_dict)
        classes_df = classes_df.append(dfa, ignore_index=True)
    return classes_df

In [ ]:
classes_df=restructure(temp_df2)
classes_df

In [ ]:
training_df = classes_df.merge(temp_df2.drop(['segmentation', 'class'], axis=1), on='id')
training_df = training_df.drop_duplicates(subset=['id'], keep='first')
training_df

In [ ]:
training_df.to_csv('training_df.csv', index=False)

Training csv has been stored now you can use that pretty easily by just loading it.

If you got any help from this notebook kindly support by **UPVOTING**.
